In [ ]:
#import ALL THE THINGS

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import os
import feather
import datetime

from ema_workbench import (Model, 
                           RealParameter, 
                           Constant,
                           TimeSeriesOutcome,
                           perform_experiments, 
                           ema_logging, 
                           save_results,
                           perform_experiments, 
                           ema_logging)

#from ema_workbench.em_framework import samplers

from ema_workbench.connectors.vensim import VensimModel

#from ema_workbench.em_framework.evaluators import LHS, MC

import ema_workbench.analysis.pairs_plotting as pairs
import ema_workbench.analysis.plotting as emaplt

# turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

In [ ]:
#define experiment parameters
uncertainties = [
                RealParameter('a',1,3), 
                RealParameter('b',1.5,4) 
            ]

outcomes =  [
                TimeSeriesOutcome('x'), 
                TimeSeriesOutcome('y'),
                TimeSeriesOutcome('xin'),
                TimeSeriesOutcome('yin')
            ]

constants = [
                Constant('x0', 0.5),
                Constant('y0', 0.5)
            ]

In [ ]:
wd = r'./'
model = VensimModel("Brusselator", wd=wd, model_file=r'201806140107_BBSD_Br_StrogatzBrusselator.vpm')

model.uncertainties = uncertainties
model.outcomes = outcomes
model.constants = constants

In [ ]:
results = perform_experiments(model, 1000)

experiments, outcomes = results

In [ ]:
figure = emaplt.lines(results,density=u'kde') #show lines, and end state density
plt.rcParams["figure.figsize"] = (10,20)
plt.show() #show figure

In [ ]:
experiments

In [ ]:
import datetime
df_expt = pd.DataFrame(experiments) #convert experiments (array of tuples) into df
df_expt = df_expt.copy() #self-copy to circumvent errors
path = "./Data/" + str(datetime.date.today()) + "Experiments" + '.feather' #create path for saving
feather.write_dataframe(df_expt.iloc[:,0:2],path) #feather it!

In [ ]:
def export_outcomes(outcomes, location = './Data/' ):
    """takes EMA outcomes and exports all time series for each outcome of interest as a .feather dataframe for easy
    import into R. Arguments: export_outcomes(name of EMA outcomes dict, desired save location). Save location must exist,
    defaults to './Data/'.
    
    """
     
    today = datetime.date.today()
    datestr = (str(today))  
    
    keylist = list(outcomes.keys())
    
    for k in keylist:
        df_temp = pd.DataFrame(outcomes[k])
        df_temp = df_temp.copy()
        path = location + datestr + k.replace(' ','') + '.feather'
        feather.write_dataframe(df_temp,path)

    return("Done.");

In [ ]:
export_outcomes(outcomes)